<div style="
  text-align:center;
  background: linear-gradient(120deg, #0f2027, #203a43, #2c5364);
  color:#f0f4ff;
  padding:60px 40px;
  border-radius:18px;
  box-shadow:0 8px 18px rgba(0,0,0,0.35);
  line-height:1.8em;
  font-size:18px;
">

<h1 style="font-size:40px; margin-bottom:10px; color:#ffffff;">
🧬 Scientific Image Forgery Detection
</h1>

<p style="
  font-size:20px; 
  color:#b8c9ff; 
  margin-top:10px; 
  margin-bottom:25px;
  font-style:italic; 
  text-align:center;
  display:block;
  width:100%;
">
“Protecting the truth in science — one pixel at a time.”
</p>

<hr style="width:120px; border:1px solid #b8c9ff; margin:25px auto;">

<p style="max-width:850px; margin:auto; color:#e5eaff;">
In modern research, <b>image integrity defines scientific credibility</b>.  
This notebook explores <b>copy–move forgeries</b> in biomedical figures and demonstrates how  
<b>AI-based visual forensics</b> can expose hidden manipulations that threaten data integrity.  
<br><br>
A fusion of <b>exploratory analysis</b> and <b>deep learning classification</b> lays the foundation  
for future pixel-level segmentation models designed to safeguard research authenticity.
</p>

<p style="margin-top:40px; font-style:italic; color:#cfd8ff; font-size:15px;">
Developed by <b>Djamila</b> | Kaggle Notebook 2025
</p>
</div>


In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from pathlib import Path
from scipy.ndimage import uniform_filter



## 🧠 1. Dataset Overview
Before diving into forgery characteristics, let's explore the overall dataset structure —  
including class balance and image dimensions.

In [ ]:

base_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
train_auth = os.listdir(f"{base_dir}/train_images/authentic")
train_forg = os.listdir(f"{base_dir}/train_images/forged")

plt.bar(["Authentic", "Forged"], [len(train_auth), len(train_forg)], color=["green", "red"])
plt.title("Distribution des images : authentiques vs falsifiées")
plt.ylabel("Nombre d'images")
plt.show()


In [ ]:
# Display 3 examples of authentic, forged images and their corresponding masks
plt.figure(figsize=(15,10))

for i in range(3):
    # Load one authentic image
    img_auth = Image.open(os.path.join(base_dir, "train_images/authentic", train_auth[i]))
    
    # Load one forged image and its mask
    img_forg = Image.open(os.path.join(base_dir, "train_images/forged", train_forg[i]))
    mask_forg = np.load(os.path.join(base_dir, "train_masks", train_forg[i].split('.')[0] + ".npy"))
    
    # Handle multiple masks (stacked along depth)
    if mask_forg.ndim == 3:
        mask_forg = np.max(mask_forg, axis=0)
    
    # --- Display ---
    plt.subplot(3,3,3*i + 1)
    plt.imshow(img_auth)
    plt.title(f"Authentic #{i+1}")
    plt.axis("off")
    
    plt.subplot(3,3,3*i + 2)
    plt.imshow(img_forg)
    plt.title(f"Forged #{i+1}")
    plt.axis("off")
    
    plt.subplot(3,3,3*i + 3)
    plt.imshow(mask_forg, cmap="Reds")
    plt.title(f"Mask #{i+1} (Tampered)")
    plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
# Image dimensions (width vs height)
dims_auth, dims_forg = [], []
for folder, dims_list in [("authentic", dims_auth), ("forged", dims_forg)]:
    for f in os.listdir(f"{base_dir}/train_images/{folder}")[:300]:
        img = Image.open(os.path.join(base_dir, "train_images", folder, f))
        dims_list.append(img.size)

auth_df = np.array(dims_auth)
forg_df = np.array(dims_forg)

plt.figure(figsize=(10,5))
sns.scatterplot(x=auth_df[:,0], y=auth_df[:,1], label="Authentic", color="green")
sns.scatterplot(x=forg_df[:,0], y=forg_df[:,1], label="Forged", color="red")
plt.title("Image Dimensions — Width vs Height")
plt.xlabel("Width (px)")
plt.ylabel("Height (px)")
plt.legend()
plt.show()


## 🔬 2. Mask-Level Statistics
We now analyze the **tampered regions** themselves — their size, proportion,  
and frequency per image — to assess how subtle or extensive manipulations are.


In [ ]:
# Define the mask file list
mask_files = os.listdir(f"{base_dir}/train_masks")

# Forged region size (log scale)
mask_areas = []
for m in mask_files:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask = np.max(mask, axis=0) if mask.ndim == 3 else mask
    mask_areas.append(mask.sum())

plt.figure(figsize=(8,4))
sns.histplot(mask_areas, bins=50, log_scale=True, color="crimson")
plt.title("Forged Region Size Distribution (Log Scale)")
plt.xlabel("Number of Forged Pixels")
plt.ylabel("Number of Images")
plt.show()


In [ ]:
# Percentage of tampered pixels per image
mask_percent = []
for m in mask_files:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask = np.max(mask, axis=0) if mask.ndim == 3 else mask
    area = mask.sum()
    h, w = mask.shape
    mask_percent.append(area / (h*w))

plt.figure(figsize=(7,4))
sns.histplot(mask_percent, bins=30, color='orange')
plt.title("Percentage of Tampered Area per Image")
plt.xlabel("Proportion of Forged Pixels (%)")
plt.show()


## 🌍 3. Spatial Patterns
Where do falsifications occur most often?  
These visualizations highlight **global and local spatial trends**.


In [ ]:
# Global average heatmap of tampered areas
heatmap = np.zeros((512,512))
for m in mask_files:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask = np.max(mask, axis=0) if mask.ndim == 3 else mask
    mask = np.array(Image.fromarray(mask).resize((512,512)))
    heatmap += mask

plt.imshow(heatmap/len(mask_files), cmap="hot")
plt.title("Global Heatmap of Forged Regions")
plt.colorbar(label="Tampering Frequency")
plt.show()


In [ ]:
# Center positions of forged regions
centers_x, centers_y = [], []
for m in mask_files:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask = np.max(mask, axis=0) if mask.ndim == 3 else mask
    y, x = np.where(mask > 0)
    if len(x) > 0:
        centers_x.append(np.mean(x))
        centers_y.append(np.mean(y))

plt.figure(figsize=(6,6))
plt.hexbin(centers_x, centers_y, gridsize=50, cmap='inferno')
plt.title("Spatial Distribution of Tampered Region Centers")
plt.xlabel("x (width)")
plt.ylabel("y (height)")
plt.colorbar(label="Frequency")
plt.show()


## 🎨 4. Geometric and Visual Characteristics
We now analyze the **shape** and **brightness** of the forged areas  
to understand if visual properties differ between authentic and manipulated figures.


In [ ]:
# Width/height ratio of forged areas
ratios = []
for m in mask_files:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask = np.max(mask, axis=0) if mask.ndim == 3 else mask
    y, x = np.where(mask > 0)
    if len(x) > 0:
        h = y.max() - y.min() + 1
        w = x.max() - x.min() + 1
        ratios.append(w/h)

plt.figure(figsize=(7,4))
sns.histplot(ratios, bins=40, color="teal")
plt.title("Width-to-Height Ratio of Tampered Regions")
plt.xlabel("Width / Height Ratio")
plt.show()


In [ ]:
# Average brightness (luminance) comparison
def get_luminance(img_path):
    img = Image.open(img_path).convert("L")
    return np.mean(img), np.std(img)

auth_means, auth_stds = zip(*[get_luminance(os.path.join(base_dir, "train_images/authentic", f))
                              for f in os.listdir(f"{base_dir}/train_images/authentic")[:300]])

forg_means, forg_stds = zip(*[get_luminance(os.path.join(base_dir, "train_images/forged", f))
                              for f in os.listdir(f"{base_dir}/train_images/forged")[:300]])

plt.figure(figsize=(6,5))
sns.kdeplot(auth_means, label="Authentic", color="green")
sns.kdeplot(forg_means, label="Forged", color="red")
plt.title("Brightness Distribution — Authentic vs Forged")
plt.xlabel("Average Brightness")
plt.legend()
plt.show()


## 🔗 5. Cross-Feature Relationship
Finally, let's check whether **brightness** correlates with **forgery size** —  
to see if exposure or tone impacts manipulation scale.


In [ ]:
from PIL import Image
import numpy as np
import os

def get_luminance(img_path):
    img = Image.open(img_path).convert("L")
    return np.mean(img), np.std(img)

# Average brightness for authentic and forged images
auth_means, _ = zip(*[get_luminance(os.path.join(base_dir, "train_images/authentic", f))
                      for f in os.listdir(f"{base_dir}/train_images/authentic")[:300]])

forg_means, _ = zip(*[get_luminance(os.path.join(base_dir, "train_images/forged", f))
                      for f in os.listdir(f"{base_dir}/train_images/forged")[:300]])

df_patterns = pd.DataFrame({
    "mask_area": mask_areas[:len(forg_means)],
    "luminosity": forg_means[:len(mask_areas)]
})

sns.scatterplot(x="luminosity", y="mask_area", data=df_patterns)
plt.title("Brightness vs Forged Area Size")
plt.xlabel("Average Brightness")
plt.ylabel("Forged Area (pixels)")
plt.show()


In [ ]:
mask_sizes = []
mask_files = os.listdir(f"{base_dir}/train_masks")

for m in mask_files[:1000]:
    mask = np.load(os.path.join(base_dir, "train_masks", m))
    mask_sizes.append(mask.sum())

plt.figure(figsize=(8,4))
sns.histplot(mask_sizes, bins=40, log_scale=True, color='crimson')
plt.title("Distribution de la taille des zones falsifiées (en pixels)")
plt.xlabel("Surface falsifiée (échelle log)")
plt.show()


## 🔍 Pattern 6 — Directional Inconsistency Analysis (DIC Map)

This pattern focuses on **detecting subtle structural inconsistencies** in forged images by analyzing the **local gradient orientation coherence**.  
In authentic images, neighboring pixels usually follow smooth and consistent gradient directions, while in manipulated areas, the edges often display **abrupt directional changes** or unnatural texture transitions.  
By computing a *Directional Inconsistency Coefficient (DIC)* using Sobel or Scharr filters, we can visualize and quantify the variance of local edge orientations — highlighting zones that deviate from natural image statistics.


In [ ]:

from pathlib import Path
import cv2
from scipy.ndimage import uniform_filter

# --- Fonction principale ---
def directional_incoherence(img_path, window_size=9):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3,3), 0)

    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)

    theta = np.arctan2(gy, gx)
    mean_theta = uniform_filter(theta, size=window_size)
    mean_theta2 = uniform_filter(theta**2, size=window_size)
    var_theta = mean_theta2 - mean_theta**2

    dic = cv2.normalize(var_theta, None, 0, 1, cv2.NORM_MINMAX)
    return dic


# --- Exemple visuel ---
auth_dir = f"{base_dir}/train_images/authentic"
forg_dir = f"{base_dir}/train_images/forged"

auth_example = Path(auth_dir) / train_auth[0]
forg_example = Path(forg_dir) / train_forg[0]

dic_auth = directional_incoherence(str(auth_example))
dic_forg = directional_incoherence(str(forg_example))

plt.figure(figsize=(12,6))
plt.subplot(2,2,1); plt.imshow(cv2.imread(str(auth_example))[:,:,::-1]); plt.title("Authentic - Image"); plt.axis("off")
plt.subplot(2,2,2); plt.imshow(dic_auth, cmap="inferno"); plt.title("Authentic - DIC map"); plt.axis("off")

plt.subplot(2,2,3); plt.imshow(cv2.imread(str(forg_example))[:,:,::-1]); plt.title("Forged - Image"); plt.axis("off")
plt.subplot(2,2,4); plt.imshow(dic_forg, cmap="inferno"); plt.title("Forged - DIC map"); plt.axis("off")
plt.show()


# --- Statistiques globales ---
def compute_dic_stats(img_folder, file_list, n=20):
    vals = []
    for f in file_list[:n]:
        try:
            dic = directional_incoherence(os.path.join(img_folder, f))
            vals.append(dic.mean())
        except Exception as e:
            print(f"Erreur sur {f}: {e}")
            continue
    return np.mean(vals), np.std(vals)

mean_auth, std_auth = compute_dic_stats(auth_dir, train_auth)
mean_forg, std_forg = compute_dic_stats(forg_dir, train_forg)

print(f"Authentic → DIC moyen: {mean_auth:.4f} ± {std_auth:.4f}")
print(f"Forged    → DIC moyen: {mean_forg:.4f} ± {std_forg:.4f}")


### 🧠 Insight

The DIC maps reveal the **spatial coherence of edges** across the image.  
Authentic regions typically exhibit uniform orientation patterns (low variance), while forged areas show **disrupted gradient flows** or **high-frequency anomalies**.  
Although global averages may appear similar, analyzing local DIC variance or combining the DIC maps with deep feature embeddings (e.g., DINOv2 or CNN outputs) can significantly enhance the detection of **localized manipulations** such as copy-move or splicing.


## 🤖 Baseline Deep Learning Model — Authentic vs Forged Classification
We train a ResNet-18 model using transfer learning to distinguish between authentic and manipulated scientific images.  
This first baseline provides a strong foundation before moving to pixel-level segmentation.


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from sklearn.model_selection import train_test_split

base_dir = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"

data = []
for folder, label in [("authentic", 0), ("forged", 1)]:
    folder_path = os.path.join(base_dir, "train_images", folder)
    for f in os.listdir(folder_path):
        if f.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
            data.append((os.path.join(folder_path, f), label))

train_data, val_data = train_test_split(data, test_size=0.2, stratify=[d[1] for d in data], random_state=42)

class ForgeryDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path, label = self.data[idx]
        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(label, dtype=torch.long)

transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = ForgeryDataset(train_data, transform=transform_train)
val_ds = ForgeryDataset(val_data, transform=transform_val)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)  # 2 classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss/len(train_loader):.4f}")

model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

print(f"Validation accuracy: {100 * correct / total:.2f}%")
